# In which I cast off the shackles of messy LaTex templates and embrace slightly less messy CSS templates.
Changing the content or styling of a resume or CV is a relatively common event that can be frustrating andtime-consuming. This post outlines the simplest possible workflow where resume content is maintained in a simple markdown file and generating `.html`, `.pdf` and `.docx` output formats is automated with a few lines of code. Styling of the outputs requires maintaining one `.css` sheet and one reference Word doc, and even if you aren't familiar with such things I can give you some basic steps for re-styling. 

**Still trying to decide if this is the workflow for you? Here are the original markdown files and the three output files for your persual**: 

[Markdown](https://sdsawtelle.github.io/attachments/resume.md) . [HTML](https://sdsawtelle.github.io/resume.html) . [PDF](files/images/post-markdown-resume/resume.pdf) . [Word Doc](files/images/post-markdown-resume/resume.docx)

# The Plan.
We'll need just two tools. `Pandoc`: the swiss army knife of interconverting file formats, and `wkhtmtopdf` the precision scalpel (?) of printing HTML to PDFs. Here is the simplest workflow:
1. Use pandoc to convert .md to .html, with styling via a .css file
2. Use `wkhtmltopdf` to print .html as .pdf
3. Use pandoc to convert .md to .docx with styling via a .docx reference file

The pdf could alternatively be generated from the markdown using pandoc with a `LaTex` engine and a `.tex` template, but it's more efficient to just borrow the styling from the HTML.

# The tl;dr
Install `Pandoc` and `wkhtmltopdf`. Now do:

In [ ]:
git clone https://github.com/sdsawtelle/markdown-resume.git
cd markdown-resume
pandoc -o resume.html -c resume-css-stylesheet.css resume.md
wkhtmltopdf resume.pdf resume.html
pandoc -o resume.docx --reference-docx=resume-docx-reference.docx resume.md

# The Flow.
## Install Pandoc and wkhtmltopdf.
If you don't already have these guys installed, then grab them from [here](https://github.com/jgm/pandoc/releases/tag/1.19) and [here](http://wkhtmltopdf.org/downloads.html). If you do have them installed, then update them. **I had to manually add the `wkhtmltopdf` executable location to PATH in order to make it play nice with Pandoc**. (I'm on Windows 10).

## Create a directory for your resume workflow.
This is the place we'll keep our markdown file and our templates -  you can get all of them by cloning the dummy repo I made for you:

In [ ]:
git clone https://github.com/sdsawtelle/markdown-resume.git

Or you can just [download the zip file](files/images/post-markdown-resume/markdown_resume.7z). You should now have a starter markdown file, a starter CSS template and a starter docx reference file. Yeah, it's all the files from my *actual* resume, because I didn't feel like editing it to make a fake one.

## Create the HTML output.
Open your command line and change to your new resume directory. Now execute:

In [ ]:
pandoc -o resume.html -c resume-css-stylesheet.css resume.md

The `-o` switch specifies the output file name, the `-c` switch specifies the css file to use for html creation and the input to the command, `resume.md`, is at the very end. 

You can inspect the resulting HTML file in your browser. Props to the guy who [originally wrote this CSS sheet](https://gist.github.com/dashed/6714393) (I've modified it only slightly), but if you're like me then you are going to immediately want to tweak some things. If you aren't familiar with CSS don't worry you can still tweak, that's what this next section is for.

### Restyling the HTML output (*for those unfamiliar with CSS*)
What pandoc has done is parse your markdown file and mapped the elements of each type (heading 1, heading 2, list etc.) to corresponding element classes in the HTML document. The CSS file is going to specify display styles for each such element class.

The first thing to do is open the HTML file in your browser, and figure out how to "inspect elements" with your browser (just do some googling). This will open a complicated looking little window for you, and by mousing over various places on the web page you will be able to view their CSS styling. Try inspecting the pink section header called "Education" - you should see some styling that looks like:

h2 {

    font-size: 24px;
    border-bottom: 1px solid #ccc;
    color: #f39;
}

This says, for example, that elements of the `h2` class in the HTML should have hexadecimal color `ff3399` which is pink! If you open the CSS file and use CTRL+f judiciously, you should be able to find where this occurrs in the document. Now try changing it to a different color, saving the changes to `.css` file, and refreshing the browser on your HTML file.

You can use this same approach to change other aspects of styling: inspect an element in your browser to see it's HTML class and styling, then open the CSS file and find the corresponding text and make your changes. 

## Create the PDF output.
In your command line execute:

In [ ]:
wkhtmltopdf resume.pdf resume.html

All our styling was already for the HTML file, woohoo! In theory you could also do `pandoc -o resume.pdf -t html5 resume.html` which would cause pandoc to call the `wkhtmltopdf` engine for the job, but I found that somehow `wkhtmltopdf` failed to resolve links in the HTML file (like the link to the .css file) unless I called it explicitly from the command line.

## Create the Word doc output
In your command line execute:

In [ ]:
pandoc -o resume.docx --reference-docx=resume-docx-reference.docx resume.md

The `--reference-docx` switch specifies a sort of "template" for styling the Word document. This template is itself a Word doc, so you can open it up in Word and start re-styling!
### Restyling the docx output
Just like it did with the HTML conversion, Pandoc has parsed your markdown file and mapped the elements of each type (heading 1, heading 2, list etc.) to corresponding element classes in the Word document. Open `resume-docx-reference.docx` in Word. Highlight the large name at the top of the document and then go to the `Home` tab and find the `Styles` section. You should see that a style called "Heading 1" is selected - that means the text you selected is of the "Heading 1" class. Right click on this style in the ribbon and click `Modify` to make all sorts of changes. Certain other changes to this reference doc will also be reflected in your output `.docx`, such as margin size. 

When you're starting from a fresh markdown file a good approach is to let pandoc make a first pass `.docx` without specifying a reference, open the resulting document and modify styles as desired, then use it as the reference for a final make. 

# Wrapping the Workflow in Shell or Python
Go ahead and combine the three commands in some kind of wrapper for ultimate easy-mode. You can also add some extra functionality in your script. For instance any time I update the resume output formats I'd like to copy the HTML file over to the directory where my blog lives so that I'm always serving the updated one. 

I *adore* python, and I hang out in IPython a lot, and I'm on Windows, so even though this is really a job for a shell script I've got a python script instead. 